In [3]:
from pathlib import Path
import cv2

sift = cv2.SIFT_create(
    nOctaveLayers=7,
    contrastThreshold=0.01,
    sigma=1.414
)


def load_logos(base_folder: str):
    logos_dict = {}
    base_path = Path(base_folder)

    for brand_folder in base_path.iterdir():
        if brand_folder.is_dir():
            brand_name = brand_folder.name
            logos = [str(file) for file in brand_folder.glob("*") if file.is_file()]
            logos_dict[brand_name] = logos

    return logos_dict

folder = "Logos" 
logos_paths = load_logos(folder)

print(logos_paths)
descriptores = {}


for brand, paths in logos_paths.items():
    descriptores[brand] = []
    for path in paths:
        logo = cv2.imread(path)
        
        if logo is None:
            print(f"Error loading image: {path}")
            continue

        gray_logo = cv2.cvtColor(logo, cv2.COLOR_BGR2GRAY)
        kp_logo, des_logo = sift.detectAndCompute(logo, None)
        descriptores[brand].append((kp_logo, des_logo))



{'Adidas': ['Logos\\Adidas\\image.png', 'Logos\\Adidas\\logo1.png'], 'Nike': ['Logos\\Nike\\logo1.png', 'Logos\\Nike\\logo10.png', 'Logos\\Nike\\logo11.png', 'Logos\\Nike\\logo12.png', 'Logos\\Nike\\logo13.jpg', 'Logos\\Nike\\logo14.png', 'Logos\\Nike\\logo15.png', 'Logos\\Nike\\logo16.png', 'Logos\\Nike\\logo17.png', 'Logos\\Nike\\logo18.png', 'Logos\\Nike\\logo19.png', 'Logos\\Nike\\logo2.png', 'Logos\\Nike\\logo20.png', 'Logos\\Nike\\logo3.png', 'Logos\\Nike\\logo4.png', 'Logos\\Nike\\logo5.png', 'Logos\\Nike\\logo6.png', 'Logos\\Nike\\logo7.png', 'Logos\\Nike\\logo9.png'], 'Puma': ['Logos\\Puma\\logo1.png', 'Logos\\Puma\\logo2.png', 'Logos\\Puma\\logo3.png', 'Logos\\Puma\\logo4.png']}


In [ ]:
import cv2
import numpy as np
from pathlib import Path

# Threshold configuration
MIN_MATCH_COUNT = 4  # Minimum matches needed for homography
BRAND_CONFIDENCE_THRESHOLD = 0  # Minimum good matches to classify as a known brand

# Load all cloth images from the Cloth folder
cloth_folder = Path("Cloth")
cloth_images = list(cloth_folder.glob("*"))

# Process each cloth image
for cloth_path in cloth_images:
    if not cloth_path.is_file():
        continue
    
    print(f"\n{'='*60}")
    print(f"Processing: {cloth_path.name}")
    print('='*60)
    
    shirt = cv2.imread(str(cloth_path), cv2.IMREAD_GRAYSCALE)
    
    if shirt is None:
        print(f"Error loading image: {cloth_path}")
        continue
    
    # Compute SIFT features directly on the shirt image
    shirt = cv2.bilateralFilter(shirt, 0, 1, 1)
    kp_shirt, des_shirt = sift.detectAndCompute(shirt, None)
    
    if des_shirt is None:
        print("No descriptors found in shirt")
        continue
    
    print(f"Found {len(kp_shirt)} SIFT keypoints")
    
    best_brand = None
    best_match_count = 0
    best_matches = None
    best_logo_kp = None
    best_H = None
    best_mask = None
    best_logo_index = None
    
    # Match against all logos from all brands
    for brand, logo_data in descriptores.items():
        for idx, (kp_logo, des_logo) in enumerate(logo_data):
            if des_logo is None:
                continue
            
            # Match with KNN
            bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
            knn_matches = bf.knnMatch(des_logo, des_shirt, k=2)
            
            # Lowe's ratio test
            good = []
            ratio_thresh = 0.6
            
            for match_pair in knn_matches:
                if len(match_pair) == 2:
                    m, n = match_pair
                    if m.distance < ratio_thresh * n.distance:
                        good.append(m)
            
            # Apply RANSAC to filter outliers if we have enough matches
            inlier_count = 0
            H = None
            mask = None
            
            if len(good) >= MIN_MATCH_COUNT:
                src_pts = np.float32([kp_logo[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
                dst_pts = np.float32([kp_shirt[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
                H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
                
                # Count only inliers (matches that fit the homography model)
                if mask is not None:
                    inlier_count = mask.sum()
            
            # Check if this is the best match so far (using inlier count)
            if inlier_count > best_match_count:
                best_match_count = inlier_count
                best_brand = brand
                best_matches = good
                best_logo_kp = kp_logo
                best_logo_index = idx
                best_H = H
                best_mask = mask
    
    # Determine if it's a known brand or unknown based on threshold
    if best_match_count < BRAND_CONFIDENCE_THRESHOLD:
        print(f"RESULT: UNKNOWN BRAND (only {best_match_count} matches found, threshold is {BRAND_CONFIDENCE_THRESHOLD})")
        is_known_brand = False
    else:
        print(f"RESULT: {best_brand} with {best_match_count} good matches ✓")
        is_known_brand = True
    
    # Display results for this cloth
    if best_matches and best_match_count >= MIN_MATCH_COUNT and is_known_brand:
        # Load the best matching logo for visualization
        logo_path = logos_paths[best_brand][best_logo_index]
        logo = cv2.imread(logo_path, cv2.IMREAD_GRAYSCALE)
        
        matchesMask = best_mask.ravel().tolist() if best_mask is not None else None
        
        # Draw matches
        result = cv2.drawMatches(
            logo, best_logo_kp,
            shirt, kp_shirt,
            best_matches, None,
            matchesMask=matchesMask,
            flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
        )
        
        cv2.namedWindow(f"Matches - {cloth_path.name}", cv2.WINDOW_NORMAL)
        cv2.imshow(f"Matches - {cloth_path.name}", result)
        cv2.resizeWindow(f"Matches - {cloth_path.name}", 1000, 1000)
        
        # Draw bounding box on original shirt image
        if best_H is not None:
            h, w = logo.shape
            logo_corners = np.float32([
                [0, 0],
                [0, h - 1],
                [w - 1, h - 1],
                [w - 1, 0]
            ]).reshape(-1, 1, 2)
            
            projected_corners = cv2.perspectiveTransform(logo_corners, best_H)
            
            shirt_color = cv2.cvtColor(shirt, cv2.COLOR_GRAY2BGR)
            shirt_with_box = cv2.polylines(
                shirt_color,
                [np.int32(projected_corners)],
                isClosed=True,
                color=(0, 255, 0),
                thickness=3,
                lineType=cv2.LINE_AA
            )
            
            # Add brand label on the image
            cv2.putText(
                shirt_with_box,
                f"Brand: {best_brand}",
                (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (0, 255, 0),
                2,
                cv2.LINE_AA
            )
            
            cv2.namedWindow(f"Detection - {cloth_path.name}", cv2.WINDOW_NORMAL)
            cv2.imshow(f"Detection - {cloth_path.name}", shirt_with_box)
            cv2.resizeWindow(f"Detection - {cloth_path.name}", 800, 800)
        
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    elif not is_known_brand:
        print(f"Skipping visualization - classified as UNKNOWN BRAND")
    else:
        print(f"Not enough matches found for detection visualization")


Processing: a1.png
Found 6329 SIFT keypoints
RESULT: Puma with 21 good matches ✓

Processing: a2.png
Found 47129 SIFT keypoints
RESULT: Adidas with 5 good matches ✓

Processing: a3.png
Found 1962 SIFT keypoints
RESULT: Puma with 26 good matches ✓

Processing: a4.png
Found 5164 SIFT keypoints
RESULT: Puma with 18 good matches ✓

Processing: adidas_shirt1.png
Found 3703 SIFT keypoints
RESULT: Adidas with 37 good matches ✓

Processing: nike_shirt1.png
Found 3160 SIFT keypoints
RESULT: Nike with 28 good matches ✓

Processing: nike_shirt2.png
Found 30394 SIFT keypoints
RESULT: Nike with 67 good matches ✓

Processing: nike_shirt3.png
Found 6144 SIFT keypoints
RESULT: Nike with 23 good matches ✓

Processing: nike_shirt5.png
Found 18009 SIFT keypoints
RESULT: Nike with 65 good matches ✓

Processing: nike_shirt6.png
Found 29973 SIFT keypoints
RESULT: Nike with 8 good matches ✓

Processing: nike_shoe1.png
Found 7304 SIFT keypoints
RESULT: Nike with 8 good matches ✓

Processing: nike_shoe2.png
F

In [ ]:
import cv2
import numpy as np
from pathlib import Path

# Threshold configuration
MIN_MATCH_COUNT = 4  # Minimum matches needed for homography
BRAND_CONFIDENCE_THRESHOLD = 0  # Minimum good matches to classify as a known brand

# Contour extraction parameters
MIN_CONTOUR_AREA = 500  # Minimum area for a contour to be considered
MAX_CONTOUR_AREA = 50000  # Maximum area for a contour to be considered
MIN_ASPECT_RATIO = 0.3  # Minimum width/height ratio
MAX_ASPECT_RATIO = 3.0  # Maximum width/height ratio

def extract_logo_regions(image):
    """
    Extract suspected logo regions using edge detection and contour analysis.
    Returns list of (x, y, w, h, roi) tuples for each suspected logo region.
    """
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    
    # Apply Canny edge detection
    edges = cv2.Canny(blurred, 50, 150)
    
    # Morphological operations to close gaps in edges
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)
    dilated = cv2.dilate(closed, kernel, iterations=2)
    
    # Find contours
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    logo_regions = []
    
    for contour in contours:
        # Calculate contour properties
        area = cv2.contourArea(contour)
        
        # Filter by area
        if area < MIN_CONTOUR_AREA or area > MAX_CONTOUR_AREA:
            continue
        
        # Get bounding rectangle
        x, y, w, h = cv2.boundingRect(contour)
        
        # Filter by aspect ratio (reject very elongated shapes)
        aspect_ratio = w / float(h)
        if aspect_ratio < MIN_ASPECT_RATIO or aspect_ratio > MAX_ASPECT_RATIO:
            continue
        
        # Add padding around the region
        padding = 10
        x_padded = max(0, x - padding)
        y_padded = max(0, y - padding)
        w_padded = min(image.shape[1] - x_padded, w + 2 * padding)
        h_padded = min(image.shape[0] - y_padded, h + 2 * padding)
        
        # Extract the region of interest
        roi = image[y_padded:y_padded + h_padded, x_padded:x_padded + w_padded]
        
        if roi.size > 0:
            logo_regions.append((x_padded, y_padded, w_padded, h_padded, roi))
    
    return logo_regions


# Load all cloth images from the Cloth folder
cloth_folder = Path("Cloth")
cloth_images = list(cloth_folder.glob("*"))

# Load logo descriptors (assuming these are pre-computed)
# descriptores = {...}  # Your existing logo descriptors dictionary
# logos_paths = {...}   # Your existing logo paths dictionary

# Process each cloth image
for cloth_path in cloth_images:
    if not cloth_path.is_file():
        continue
    
    print(f"\n{'='*60}")
    print(f"Processing: {cloth_path.name}")
    print('='*60)
    
    shirt = cv2.imread(str(cloth_path), cv2.IMREAD_GRAYSCALE)
    
    if shirt is None:
        print(f"Error loading image: {cloth_path}")
        continue
    
    # Extract suspected logo regions using edge detection and contours
    print("Extracting logo regions...")
    logo_regions = extract_logo_regions(shirt)
    print(f"Found {len(logo_regions)} suspected logo regions")
    
    # Visualize the extracted regions
    if len(logo_regions) > 0:
        # Create visualization of edge detection and contours
        shirt_color = cv2.cvtColor(shirt, cv2.COLOR_GRAY2BGR)
        blurred = cv2.GaussianBlur(shirt, (5, 5), 0)
        edges = cv2.Canny(blurred, 50, 150)
        edges_color = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
        
        # Draw rectangles on detected regions
        shirt_with_regions = shirt_color.copy()
        for idx, (x, y, w, h, roi) in enumerate(logo_regions):
            cv2.rectangle(shirt_with_regions, (x, y), (x + w, y + h), (0, 255, 255), 2)
            cv2.putText(shirt_with_regions, f"R{idx+1}", (x, y-5), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        
        # Show edge detection result
        cv2.namedWindow(f"Edges - {cloth_path.name}", cv2.WINDOW_NORMAL)
        cv2.imshow(f"Edges - {cloth_path.name}", edges_color)
        cv2.resizeWindow(f"Edges - {cloth_path.name}", 800, 800)
        
        # Show detected regions on original image
        cv2.namedWindow(f"Detected Regions - {cloth_path.name}", cv2.WINDOW_NORMAL)
        cv2.imshow(f"Detected Regions - {cloth_path.name}", shirt_with_regions)
        cv2.resizeWindow(f"Detected Regions - {cloth_path.name}", 800, 800)
        
        # Show each extracted ROI separately
        for idx, (x, y, w, h, roi) in enumerate(logo_regions):
            cv2.namedWindow(f"ROI {idx+1} - {cloth_path.name}", cv2.WINDOW_NORMAL)
            cv2.imshow(f"ROI {idx+1} - {cloth_path.name}", roi)
            cv2.resizeWindow(f"ROI {idx+1} - {cloth_path.name}", 300, 300)
        
        print("\nPress any key to continue with SIFT matching...")
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    if len(logo_regions) == 0:
        print("No logo regions detected - trying full image instead")
        logo_regions = [(0, 0, shirt.shape[1], shirt.shape[0], shirt)]
    
    best_brand = None
    best_match_count = 0
    best_matches = None
    best_logo_kp = None
    best_H = None
    best_mask = None
    best_logo_index = None
    best_region = None
    best_region_coords = None
    best_roi_kp = None
    
    # Process each suspected logo region
    for region_idx, (x, y, w, h, roi) in enumerate(logo_regions):
        print(f"\n  Analyzing region {region_idx + 1}/{len(logo_regions)} at ({x}, {y}, {w}, {h})")
        
        # Compute SIFT features on this region
        kp_roi, des_roi = sift.detectAndCompute(roi, None)
        
        if des_roi is None:
            print(f"    No descriptors found in region {region_idx + 1}")
            continue
        
        print(f"    Found {len(kp_roi)} SIFT keypoints in this region")
        
        # Match against all logos from all brands
        for brand, logo_data in descriptores.items():
            for idx, (kp_logo, des_logo) in enumerate(logo_data):
                if des_logo is None:
                    continue
                
                # Match with KNN
                bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
                knn_matches = bf.knnMatch(des_logo, des_roi, k=2)
                
                # Lowe's ratio test
                good = []
                ratio_thresh = 0.6
                
                for match_pair in knn_matches:
                    if len(match_pair) == 2:
                        m, n = match_pair
                        if m.distance < ratio_thresh * n.distance:
                            good.append(m)
                
                # Apply RANSAC to filter outliers if we have enough matches
                inlier_count = 0
                H = None
                mask = None
                
                if len(good) >= MIN_MATCH_COUNT:
                    src_pts = np.float32([kp_logo[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
                    dst_pts = np.float32([kp_roi[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
                    
                    # Adjust destination points to global coordinates
                    dst_pts_global = dst_pts.copy()
                    dst_pts_global[:, 0, 0] += x
                    dst_pts_global[:, 0, 1] += y
                    
                    H, mask = cv2.findHomography(src_pts, dst_pts_global, cv2.RANSAC, 5.0)
                    
                    # Count only inliers (matches that fit the homography model)
                    if mask is not None:
                        inlier_count = int(mask.sum())
                
                # Check if this is the best match so far (using inlier count)
                if inlier_count > best_match_count:
                    best_match_count = inlier_count
                    best_brand = brand
                    best_matches = good
                    best_logo_kp = kp_logo
                    best_logo_index = idx
                    best_region = roi
                    best_region_coords = (x, y, w, h)
                    best_H = H
                    best_mask = mask
                    
                    # Store keypoints in global coordinates for visualization
                    kp_roi_global = []
                    for kp in kp_roi:
                        kp_copy = cv2.KeyPoint(kp.pt[0] + x, kp.pt[1] + y, kp.size, 
                                               kp.angle, kp.response, kp.octave, kp.class_id)
                        kp_roi_global.append(kp_copy)
                    best_roi_kp = kp_roi_global
    
    # Determine if it's a known brand or unknown based on threshold
    if best_match_count < BRAND_CONFIDENCE_THRESHOLD:
        print(f"\nRESULT: UNKNOWN BRAND (only {best_match_count} matches found, threshold is {BRAND_CONFIDENCE_THRESHOLD})")
        is_known_brand = False
    else:
        print(f"\nRESULT: {best_brand} with {best_match_count} good matches ✓")
        if best_region_coords:
            print(f"Logo detected in region: {best_region_coords}")
        is_known_brand = True
    
    # Display results for this cloth
    if best_matches and best_match_count >= MIN_MATCH_COUNT and is_known_brand:
        # Load the best matching logo for visualization
        logo_path = logos_paths[best_brand][best_logo_index]
        logo = cv2.imread(logo_path, cv2.IMREAD_GRAYSCALE)
        
        matchesMask = best_mask.ravel().tolist() if best_mask is not None else None
        
        # Draw matches
        result = cv2.drawMatches(
            logo, best_logo_kp,
            shirt, best_roi_kp,
            best_matches, None,
            matchesMask=matchesMask,
            flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
        )
        
        cv2.namedWindow(f"Matches - {cloth_path.name}", cv2.WINDOW_NORMAL)
        cv2.imshow(f"Matches - {cloth_path.name}", result)
        cv2.resizeWindow(f"Matches - {cloth_path.name}", 1000, 1000)
        
        # Draw bounding box on original shirt image
        if best_H is not None:
            h_logo, w_logo = logo.shape
            logo_corners = np.float32([
                [0, 0],
                [0, h_logo - 1],
                [w_logo - 1, h_logo - 1],
                [w_logo - 1, 0]
            ]).reshape(-1, 1, 2)
            
            projected_corners = cv2.perspectiveTransform(logo_corners, best_H)
            
            shirt_color = cv2.cvtColor(shirt, cv2.COLOR_GRAY2BGR)
            
            # Draw the detected region rectangle (in blue)
            if best_region_coords:
                x, y, w, h = best_region_coords
                cv2.rectangle(shirt_color, (x, y), (x + w, y + h), (255, 0, 0), 2)
            
            # Draw the logo detection bounding box (in green)
            shirt_with_box = cv2.polylines(
                shirt_color,
                [np.int32(projected_corners)],
                isClosed=True,
                color=(0, 255, 0),
                thickness=3,
                lineType=cv2.LINE_AA
            )
            
            # Add brand label on the image
            cv2.putText(
                shirt_with_box,
                f"Brand: {best_brand}",
                (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (0, 255, 0),
                2,
                cv2.LINE_AA
            )
            
            cv2.namedWindow(f"Detection - {cloth_path.name}", cv2.WINDOW_NORMAL)
            cv2.imshow(f"Detection - {cloth_path.name}", shirt_with_box)
            cv2.resizeWindow(f"Detection - {cloth_path.name}", 800, 800)
        
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    elif not is_known_brand:
        print(f"Skipping visualization - classified as UNKNOWN BRAND")
    else:
        print(f"Not enough matches found for detection visualization")


Processing: a1.png
Extracting logo regions...
Found 2 suspected logo regions

Press any key to continue with SIFT matching...

  Analyzing region 1/2 at (159, 298, 221, 124)
    Found 176 SIFT keypoints in this region

  Analyzing region 2/2 at (25, 0, 453, 697)
    Found 421 SIFT keypoints in this region

RESULT: Puma with 16 good matches ✓
Logo detected in region: (159, 298, 221, 124)

Processing: a2.png
Extracting logo regions...
Found 1 suspected logo regions

Press any key to continue with SIFT matching...

  Analyzing region 1/1 at (707, 1757, 111, 243)
    Found 13 SIFT keypoints in this region

RESULT: Nike with 4 good matches ✓
Logo detected in region: (707, 1757, 111, 243)

Processing: a3.png
Extracting logo regions...
Found 8 suspected logo regions

Press any key to continue with SIFT matching...

  Analyzing region 1/8 at (689, 845, 39, 50)
    Found 12 SIFT keypoints in this region

  Analyzing region 2/8 at (161, 424, 60, 127)
    Found 2 SIFT keypoints in this region

 

In [ ]:
import cv2
import numpy as np
from pathlib import Path

# Threshold configuration
MIN_MATCH_COUNT = 4  # Minimum matches needed for homography
BRAND_CONFIDENCE_THRESHOLD = 0  # Minimum good matches to classify as a known brand

def preprocess_image_with_segmentation(image):
    """
    Preprocess image using color segmentation and Canny edge detection.
    Returns a processed image suitable for SIFT feature detection.
    """
    # Convert to color for segmentation
    if len(image.shape) == 2:
        image_color = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    else:
        image_color = image.copy()
    
    # Apply bilateral filter to reduce noise while preserving edges
    #filtered = cv2.bilateralFilter(image_color, 9, 75, 75)
    filtered = image_color
    # Convert to different color spaces for better segmentation
    hsv = cv2.cvtColor(filtered, cv2.COLOR_BGR2HSV)
    lab = cv2.cvtColor(filtered, cv2.COLOR_BGR2LAB)
    
    # Perform K-means clustering for color segmentation
    # Reshape image to be a list of pixels
    pixels = filtered.reshape((-1, 3))
    pixels = np.float32(pixels)
    
    # Define criteria and apply K-means
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    k = 5  # Number of clusters
    _, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_PP_CENTERS)
    
    # Convert back to 8-bit values
    centers = np.uint8(centers)
    segmented = centers[labels.flatten()]
    segmented_image = segmented.reshape(image_color.shape)
    
    # Convert segmented image to grayscale for edge detection
    segmented_gray = cv2.cvtColor(segmented_image, cv2.COLOR_BGR2GRAY)
    
    # Apply Canny edge detection on the segmented image
    edges = cv2.Canny(segmented_gray, 50, 150)
    
    # Dilate edges slightly to make them more prominent
    kernel = np.ones((2, 2), np.uint8)
    edges_dilated = cv2.dilate(edges, kernel, iterations=1)
    
    # Create the final processed image
    # For each edge pixel in the Canny output, keep the corresponding pixel from segmented image
    processed = segmented_gray.copy()
    
    # Enhance edge regions by blending
    # Where there's an edge, use the original segmented value
    # Where there's no edge, slightly blur to reduce noise
    mask_edges = edges_dilated > 0
    
    # Apply Gaussian blur to non-edge regions
    blurred = cv2.GaussianBlur(segmented_gray, (5, 5), 0)
    processed[~mask_edges] = blurred[~mask_edges]
    
    # Enhance contrast
    processed = cv2.equalizeHist(processed)
    
    # Combine edge information with segmented image
    # Give more weight to edge pixels
    alpha = 0.5
    final = cv2.addWeighted(processed, alpha, edges_dilated, 1 - alpha, 0)
    
    return final, segmented_image, edges


# Load all cloth images from the Cloth folder
cloth_folder = Path("Cloth")
cloth_images = list(cloth_folder.glob("*"))

# Process each cloth image
for cloth_path in cloth_images:
    if not cloth_path.is_file():
        continue
    
    print(f"\n{'='*60}")
    print(f"Processing: {cloth_path.name}")
    print('='*60)
    
    # Load image in color first
    shirt_color = cv2.imread(str(cloth_path))
    
    if shirt_color is None:
        print(f"Error loading image: {cloth_path}")
        continue
    
    # Preprocess with segmentation and edge detection
    print("Applying color segmentation and edge detection...")
    processed_shirt, segmented, edges = preprocess_image_with_segmentation(shirt_color)
    
    # Compute SIFT features on the processed image
    kp_shirt, des_shirt = sift.detectAndCompute(processed_shirt, None)
    
    if des_shirt is None:
        print("No descriptors found in shirt")
        continue
    
    print(f"Found {len(kp_shirt)} SIFT keypoints")
    
    best_brand = None
    best_match_count = 0
    best_matches = None
    best_logo_kp = None
    best_H = None
    best_mask = None
    best_logo_index = None
    
    # Match against all logos from all brands
    for brand, logo_data in descriptores.items():
        for idx, (kp_logo, des_logo) in enumerate(logo_data):
            if des_logo is None:
                continue
            
            # Match with KNN
            bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
            knn_matches = bf.knnMatch(des_logo, des_shirt, k=2)
            
            # Lowe's ratio test
            good = []
            ratio_thresh = 0.6
            
            for match_pair in knn_matches:
                if len(match_pair) == 2:
                    m, n = match_pair
                    if m.distance < ratio_thresh * n.distance:
                        good.append(m)
            
            # Apply RANSAC to filter outliers if we have enough matches
            inlier_count = 0
            H = None
            mask = None
            
            if len(good) >= MIN_MATCH_COUNT:
                src_pts = np.float32([kp_logo[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
                dst_pts = np.float32([kp_shirt[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
                H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
                
                # Count only inliers (matches that fit the homography model)
                if mask is not None:
                    inlier_count = int(mask.sum())
            
            # Check if this is the best match so far (using inlier count)
            if inlier_count > best_match_count:
                best_match_count = inlier_count
                best_brand = brand
                best_matches = good
                best_logo_kp = kp_logo
                best_logo_index = idx
                best_H = H
                best_mask = mask
    
    # Determine if it's a known brand or unknown based on threshold
    if best_match_count < BRAND_CONFIDENCE_THRESHOLD:
        print(f"RESULT: UNKNOWN BRAND (only {best_match_count} matches found, threshold is {BRAND_CONFIDENCE_THRESHOLD})")
        is_known_brand = False
    else:
        print(f"RESULT: {best_brand} with {best_match_count} good matches ✓")
        is_known_brand = True
    
    # Display preprocessing results
    preprocessing_stack = np.hstack([
        cv2.cvtColor(cv2.resize(cv2.cvtColor(shirt_color, cv2.COLOR_BGR2GRAY), (300, 300)), cv2.COLOR_GRAY2BGR),
        cv2.resize(segmented, (300, 300)),
        cv2.cvtColor(cv2.resize(edges, (300, 300)), cv2.COLOR_GRAY2BGR),
        cv2.cvtColor(cv2.resize(processed_shirt, (300, 300)), cv2.COLOR_GRAY2BGR)
    ])
    
    cv2.namedWindow(f"Preprocessing - {cloth_path.name}", cv2.WINDOW_NORMAL)
    cv2.imshow(f"Preprocessing - {cloth_path.name}", preprocessing_stack)
    cv2.resizeWindow(f"Preprocessing - {cloth_path.name}", 1200, 300)
    
    # Add labels to the preprocessing display
    labels = ["Original", "Segmented", "Edges", "Processed"]
    for i, label in enumerate(labels):
        cv2.putText(preprocessing_stack, label, (i * 300 + 10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    # Display results for this cloth
    if best_matches and best_match_count >= MIN_MATCH_COUNT and is_known_brand:
        # Load the best matching logo for visualization
        logo_path = logos_paths[best_brand][best_logo_index]
        logo = cv2.imread(logo_path, cv2.IMREAD_GRAYSCALE)
        
        matchesMask = best_mask.ravel().tolist() if best_mask is not None else None
        
        # Draw matches
        result = cv2.drawMatches(
            logo, best_logo_kp,
            processed_shirt, kp_shirt,
            best_matches, None,
            matchesMask=matchesMask,
            flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
        )
        
        cv2.namedWindow(f"Matches - {cloth_path.name}", cv2.WINDOW_NORMAL)
        cv2.imshow(f"Matches - {cloth_path.name}", result)
        cv2.resizeWindow(f"Matches - {cloth_path.name}", 1000, 1000)
        
        # Draw bounding box on original shirt image
        if best_H is not None:
            h, w = logo.shape
            logo_corners = np.float32([
                [0, 0],
                [0, h - 1],
                [w - 1, h - 1],
                [w - 1, 0]
            ]).reshape(-1, 1, 2)
            
            projected_corners = cv2.perspectiveTransform(logo_corners, best_H)
            
            shirt_with_box = cv2.polylines(
                shirt_color.copy(),
                [np.int32(projected_corners)],
                isClosed=True,
                color=(0, 255, 0),
                thickness=3,
                lineType=cv2.LINE_AA
            )
            
            # Add brand label on the image
            cv2.putText(
                shirt_with_box,
                f"Brand: {best_brand}",
                (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (0, 255, 0),
                2,
                cv2.LINE_AA
            )
            
            cv2.namedWindow(f"Detection - {cloth_path.name}", cv2.WINDOW_NORMAL)
            cv2.imshow(f"Detection - {cloth_path.name}", shirt_with_box)
            cv2.resizeWindow(f"Detection - {cloth_path.name}", 800, 800)
        
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    elif not is_known_brand:
        print(f"Skipping visualization - classified as UNKNOWN BRAND")
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print(f"Not enough matches found for detection visualization")
        cv2.waitKey(0)
        cv2.destroyAllWindows()